In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<h4>Read dataset</h4>

In [3]:
client_train = pd.read_csv("data/client_train.csv")
invoice_train = pd.read_csv("data/invoice_train.csv", low_memory = False)
client_test = pd.read_csv("data/client_test.csv")
invoice_test = pd.read_csv("data/invoice_test.csv", low_memory = False)
client_test_target_proba = pd.read_csv("data/client_test_target.csv")

Assume fraud when probability of fraud >= 0.5 for test dataset

In [4]:
client_test = client_test.merge(client_test_target_proba, on = "client_id", how = "inner")
client_test["target"] = client_test["target"].apply(lambda x: 1 if x >= 0.5 else 0)

client_id is the unique identifier that links invoice and client dataset together.<br>
Perform inner join on them to obtain train and test dataset

In [5]:
train = invoice_train.merge(client_train, on = "client_id", how = "inner")
test = invoice_test.merge(client_test, on = "client_id", how = "inner")

<h4>Start of EDA</h4>

Inspecting train data.

In [6]:
train

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,...,consommation_level_4,old_index,new_index,months_number,counter_type,disrict,client_catg,region,creation_date,target
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,...,0,14302,14384,4,ELEC,60,11,101,31/12/1994,0.0
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,...,0,12294,13678,4,ELEC,60,11,101,31/12/1994,0.0
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,...,0,14624,14747,4,ELEC,60,11,101,31/12/1994,0.0
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,...,0,14747,14849,4,ELEC,60,11,101,31/12/1994,0.0
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,...,0,15066,15638,12,ELEC,60,11,101,31/12/1994,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4476744,train_Client_99998,2005-08-19,10,1253571,0,202,9,1,400,135,...,0,3197,3732,8,ELEC,60,11,101,22/12/1993,0.0
4476745,train_Client_99998,2005-12-19,10,1253571,0,202,6,1,200,6,...,0,3732,3938,4,ELEC,60,11,101,22/12/1993,0.0
4476746,train_Client_99999,1996-09-25,11,560948,0,203,6,1,259,0,...,0,13884,14143,4,ELEC,60,11,101,18/02/1986,0.0
4476747,train_Client_99999,1996-05-28,11,560948,0,203,6,1,603,0,...,0,13281,13884,4,ELEC,60,11,101,18/02/1986,0.0


Inspecting test data.

In [7]:
test

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,...,consommation_level_4,old_index,new_index,months_number,counter_type,disrict,client_catg,region,creation_date,target
0,test_Client_0,2018-03-16,11,651208,0,203,8,1,755,0,...,0,19145,19900,8,ELEC,62,11,307,28/05/2002,1
1,test_Client_0,2014-03-21,11,651208,0,203,8,1,1067,0,...,0,13725,14792,8,ELEC,62,11,307,28/05/2002,1
2,test_Client_0,2014-07-17,11,651208,0,203,8,1,0,0,...,0,14792,14792,4,ELEC,62,11,307,28/05/2002,1
3,test_Client_0,2015-07-13,11,651208,0,203,9,1,410,0,...,0,16122,16532,4,ELEC,62,11,307,28/05/2002,1
4,test_Client_0,2016-07-19,11,651208,0,203,9,1,412,0,...,0,17471,17883,4,ELEC,62,11,307,28/05/2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1939725,test_Client_9999,2010-03-11,11,869269,0,203,6,1,248,0,...,0,21114,21362,4,ELEC,69,11,104,14/03/1990,1
1939726,test_Client_9999,2011-03-15,11,869269,0,203,6,1,260,0,...,0,21362,21622,4,ELEC,69,11,104,14/03/1990,1
1939727,test_Client_9999,2012-12-03,11,869269,0,203,6,1,312,0,...,0,22272,22584,4,ELEC,69,11,104,14/03/1990,1
1939728,test_Client_9999,2009-07-01,11,869269,0,203,6,1,236,0,...,0,19909,20145,4,ELEC,69,11,104,14/03/1990,1


Confirm that the tables had one-to-one relationship

In [8]:
print(f'dimension for client_train is {client_train.shape}')
print(f'dimension for client_test is {client_test.shape}')
print(f'dimension for invoice_train is {invoice_train.shape}')
print(f'dimension for invoice_test is {invoice_test.shape}')
print(f'dimension for train is {train.shape}')
print(f'dimension for test is {test.shape}')

dimension for client_train is (135493, 6)
dimension for client_test is (58069, 6)
dimension for invoice_train is (4476749, 16)
dimension for invoice_test is (1939730, 16)
dimension for train is (4476749, 21)
dimension for test is (1939730, 21)


Concatenate to check for:<br>
1) Any duplicates across train and test set provided. <br>
2) If they share the same set of categorical variables <br>

In [9]:
train_and_test = pd.concat([train, test])
print(f'dimension for train_and_test is {train_and_test.shape}')

dimension for train_and_test is (6416479, 21)


Check for duplicated rows and NA values.<br>
No signs of overlapping data between train and test set.<br>
If there are,<br>
num of duplicated rows in train_and_test > num of duplicated rows in train + num of duplicated rows in test

In [10]:
def check_for_duplicates_and_na(df):
    print(f'Number of duplicated rows: {df.duplicated().sum()}')
    print(f'Number of NA values: {df.isna().sum().sum()}')

print('train')
check_for_duplicates_and_na(train)
print('\t')
print('test')
check_for_duplicates_and_na(test)
print('\t')
print('train and test')
check_for_duplicates_and_na(train_and_test)

train
Number of duplicated rows: 11
Number of NA values: 0
	
test
Number of duplicated rows: 8
Number of NA values: 0
	
train and test
Number of duplicated rows: 19
Number of NA values: 0


Unique counts of each column in train, test and combined to identify categorical variables
<br>
<br>
Categorical values:
1) counter_code
2) tarif_type
3) counter_coefficient
4) counter_statue
5) reading_remarque
6) counter_type
7) region
8) disrict
9) client_catg

In [11]:
def unique_count_for_columns_all_df(lst_of_df, lst_of_df_type):
    res = pd.DataFrame(
        lst_of_df[0].nunique(), 
        columns = [lst_of_df_type[0]]
        ).sort_values(by = lst_of_df_type[0], ascending = False)
    for i in range(1, len(lst_of_df)):
        right_df = pd.DataFrame(lst_of_df[i].nunique(), columns = [lst_of_df_type[i]])
        res = res.merge(right_df, left_index = True, right_index = True)
    return res

print(f'Unique count for train, test, train_test datasets')
print(unique_count_for_columns_all_df([train, test, train_and_test], 
                                      ['train', 'test', 'all']))

Unique count for train, test, train_test datasets
                       train    test     all
counter_number        201893   91966  276455
new_index             157980  118365  183299
old_index             155648  116512  180326
client_id             135493   58069  193562
consommation_level_2   12576    9700   14564
consommation_level_4   12075    8838   14114
consommation_level_1    8295    6546    9516
invoice_date            8275    6934    8697
creation_date           8088    7388    8317
consommation_level_3    2253    1862    2597
months_number           1370     914    2226
counter_code              42      39      42
region                    25      24      25
tarif_type                17      16      17
counter_coefficient       16       7      17
counter_statue            12       6      18
reading_remarque           8       4       8
disrict                    4       4       4
client_catg                3       3       3
counter_type               2       2       2
targe

Summary statistics for train dataset<br>
Problems:
1) There are months that do not lie within 1 to 12
2) There are outlier values for months_number, reading_remarque, counter_coefficient, consommation_level_1 to 4, and index

To do:
1) Look for patterns in fraudulent transactions
2) Handle the outlier values

In [12]:
pd.set_option('display.float_format', '{:,.0f}'.format)
train.describe()

,tarif_type,counter_number,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,disrict,client_catg,region,target
count,"4,476,749","4,476,749","4,476,749","4,476,749","4,476,749","4,476,749","4,476,749","4,476,749","4,476,749","4,476,749","4,476,749","4,476,749","4,476,749","4,476,749","4,476,749","4,476,749"
mean,20,"123,058,699,065",172,7,1,411,109,20,53,"17,767","18,350",45,63,12,210,0
std,13,"1,657,267,274,262",134,2,0,757,"1,220",157,875,"40,367","40,953","3,128",3,6,104,0
min,8,0,0,5,0,0,0,0,0,0,0,0,60,11,101,0
25%,11,"121,108",5,6,1,79,0,0,0,"1,791","2,056",4,62,11,103,0
50%,11,"494,561",203,8,1,274,0,0,0,"7,690","8,192",4,62,11,301,0
75%,40,"1,115,161",207,9,1,600,0,0,0,"21,660","22,343",4,63,11,309,0
max,45,"27,981,145,458,733",600,413,50,"999,910","999,073","64,492","547,946","2,800,280","2,870,972","636,624",69,51,399,1


Split train data set into fraud and not fraud to explore patterns

In [43]:
train_fraud = train[train["target"] == 1]
train_not_fraud = train[train["target"] == 0]

Distribution in months_number in train_fraud

In [47]:
train_fraud["is_valid_month"] = train_fraud["months_number"].apply(lambda x: 1 if x in list(range(1, 13)) else 0)
train_fraud["is_valid_month"].value_counts()

C:\Users\User\AppData\Local\Temp\ipykernel_27340\3819926155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_fraud["is_valid_month"] = train_fraud["months_number"].apply(lambda x: 1 if x in list(range(1, 13)) else 0)


is_valid_month
1    351345
0      1767
Name: count, dtype: int64